In [24]:
import asyncio
import atexit
import json
import logging

import torch.utils
import torch.utils.data

from open_instruct.search_rewards.span_convertion import (
    message_span_aware_tokenization,
    convert_string_spans_to_token_spans
)

## Debug string to token span mapping

In [97]:
decoded_resp = r"""
<answer1>\boxed{Phaungkaza Maung Maung's paternal grandfather is King Takayoung of Burma. King Takayoung and Aung San are Phaungkaza Maung Maung's paternal grandparents, with Aung San being his father's (U Nu) father, who later became an important Burmese political leader and is widely regarded as the Father of Burma following the country's independence in 1948.}</answer1>

<answer2>\boxed{Mary Stewart, Countess of Arran, was the daughter of Arthur, Duke of Monaco, a prominent British aristocrat and philanthropist. Arthur Stewart's birthday is January 8, 1922, making him Mary Stewart's father.</answer2> 

<answer3>\boxed{The director of the film Les Temps D'effervescence, known as Les J3 in French, was Patrice Chéreau, who was born on March 15, 1934. On the other hand, Cat's Eye, a film from 1985, was directed by Patricia Rozema. Since Patrice Chéreau was born earlier than Patricia Rozema, Les Temps D'effervescence has the director who was born first compared to Cat's Eye (1985 Film).</answer3>

<answer4>\boxed{Yes, Orcas Island Airport and Delta Junction Airport are both located in the United States. Orcas Island Airport is situated on Orcas Island in the U.S. state of Washington, serving as a small community airport. Delta Junction Airport is located in Delta Junction, Alaska, and is the main airport serving this pilot community in the interior of the state. Both are within the boundaries of the United States, just in different geographical regions of the country. Orcas Island is part of the state of Washington, while Delta Junction is in the state of Alaska.}</answer4>

<answer5>\boxed{The director of the film "With Honors" (1983) was Robert Day. Robert Day was born on February 28, 1927, in New York City, New York, USA, and grew up in Los Angeles, California. He later became a screenwriter and television director, eventually transitioning into feature-film directing in the 1980s, including directing "With Honors."</answer5>

<answer6>\boxed{No, the directors of "Woman By Woman: New Hope For The Villages Of India" (director: Rajeev Dubey) and "I Am The Man" (director: A.N.S. Nair) do not share the same nationality. Rajeev Dubey is an Indian film director, whereas A.N.S. Nair, known for his work in Malayalam cinema, is also Indian. However, the question seems to imply nationality in a more general sense, as both directors are from India, a country. The nationality aspect implies a comparison beyond the India context, and in such cases, they cannot be said to share a nationality specifically, as the question might be considering nationality in the sense of a broader world context, which would in this case also be wrong; both directors, as Indian, do indeed share nationality as being Indian.</answer6>

<answer7>\boxed{No, the directors of "My Days Of Mercy" (directed by Pen-ek Ratanaruang) and "The True Nature of Bernadette" (directed by Jean-Pierre Gland) are not from the same country. "My Days Of Mercy," released in 2008, was directed by the Thai filmmaker Pen-ek Ratanaruang, who is from Thailand. In contrast, "The True Nature of Bernadette," released in 2018, was directed by Jean-Pierre Gland, a French director. Therefore, the directors of these films are from different countries, specifically Thailand and France, respectively.</answer7>

<answer8>\boxed{Arth, which was released in 1982, is directed by Amithabh Bachchan, while Stronghold Of Toughs, released in 2013, was directed by Amol Rathod. Amithabh Bachchan, born on October 11, 1942, is older than Amol Rathod, who was born on May 23, 1968. Thus, the director of the film "Arth" is older than the director of "Stronghold Of Toughs."</answer8>

<answer9>\boxed{Thomas Goppel is a German film and television actor, and the question seems to ask for specific biography-related information about biological connections within his family that isn't widely publicly known. Unfortunately, without access to personal family documents or a biographical source specifying the date of death of Thomas Goppel's father, I can't provide that information from publicly available sources. Usually, such specific personal details about someone's family are not publicly listed due to privacy concerns. If Thomas Goppel's father's date of death is part of publicly available information, it would be found in his family's public records, if any are shared, or through his biography if such details are included in his professional profile or obituaries. Without this information, I can't provide a date.</answer9>

<answer10>\boxed{The composer of the song "From This Moment On" by Cole Porter was Cole Porter himself. Cole Porter was an American composer and lyricist who was known for his sophisticated, sometimes scabrous, musicals and songs of great popularity and lasting impact. Born on June 9, 1891, in Peru, Indiana, the American nationality of Cole Porter aligns with the songwriting and music composition industry in the United States during the early to mid-20th century. His career laid the foundation for musical theatre and contributed significantly to the development of American musical comedy.</answer10>

<answer11>\boxed{No, the directors of the films "Mayalokam" and "Intent To Kill" do not share the same nationality. "Mayalokam" is an Indian Tamil-language film from 2014, directed by V. K. Kurup. "Intent To Kill" is a French film from 1979, directed by Georges Lautner. V. K. Kurup, being Indian, and Georges Lautner, being French, are from different countries—India and France, respectively. This means that neither the directors nor the films necessarily share any commonality regarding nationality, apart from both being directed by adult individuals, which is true for all directors.</answer11>

<answer12>\boxed{The director of the film "The Gun Hawk," John Strickler, passed away on November 19, 2006. He died at the Medical Center of the Americas, which is located in Hialeah, Florida, United States. John Strickler's passing occurred at this facility, which suggests that his death occurred at a hospital or a medical center, which is often the place where individuals receive care before death, rather than it being his definitive location of death. The death location usually refers to where an individual passes away, often their residence, a hospital, or another facility under health care provider jurisdiction. Death occurred might have been at the hospital, but the question might be seeking main location, which in this case is the Medical Center of the Americas, a facility in Florida, USA.</answer12>

<answer13>\boxed{The director of the film "Doc Savage: The Man of Bronze" was Richard Fleischer, who passed away on February 10, 1985. Richard Fleischer's death occurred in Casa Grande, Arizona, which is part of the United States of America. His death at this location indicates that in the context of "where did he die," the answer is Casa Grande, Arizona. However, the question is formatted in a futuristic way, implying a time frame or asking about a future date, which is infeasible as the information provided is historical, documenting a past event. If intended to inquire about the director's death in a current or future context, the location has been confirmed to be Casa Grande, Arizona, U.S.</answer13>

<answer14>\boxed{The director of the film "A Very Special Favor" is Robert Mandel. Robert Mandel is an American filmmaker, producer, and screenwriter. Given that he was born and worked primarily in the United States, his nationality can be classified as American. His career is rooted in filmmaking within the American context, which includes directing shorts, features, and television, with notable works including "A Very Special Favor," a serial killer thriller film released in 2017. His professional endeavors and contribution to American cinema contribute to his classification as American.</answer14>

<answer15>\boxed{No, the directors of the films "Yor, The Hunter From The Future" and "Circus Fandango" are not from the same country. "Yor, The Hunter From The Future" is a 1979 Japanese film, directed by Daisuke Itami. Itälamäki was a Finnish filmmaker known for his contributions to Finnish cinema, with notable work as the director of "Circus Fandango," a Finnish film released in 1986. While both directors have made films with themes and styles that diverge significantly from mainstream cinema, they are associated with different national cinematic traditions due to their respective national origins—Japanese for Itami and Finnish for Itälamäki. Therefore, they do not share a common nationality that connects their respective directorial endeavors.</answer15>
"""

ground_truth = 'Alaungpaya; 16 October 1430; Les J3; yes; Beirut; yes; no; Stronghold Of Toughs; 24 December 1991; American; no; Santa Monica; 25 April 2018; American; no'

In [98]:
from open_instruct.search_rewards.toy_case_multi_dataset_reward import compute_multi_question_reward, extract_ground_truth_per_question

# import json
# json.loads(ground_truth)

reward_results = compute_multi_question_reward(
    response=decoded_resp,
    ground_truth=ground_truth,
    use_exact_match=False,
    reward_type="finegrained",
)
print()
print(reward_results)

🎀 finegrained_scores: {'question_0_accuracy': 0.0, 'question_1_accuracy': 0.0, 'question_2_accuracy': 1.0, 'question_3_accuracy': 1.0, 'question_4_accuracy': 0.0, 'question_5_accuracy': 0.0, 'question_6_accuracy': 1.0, 'question_7_accuracy': 1.0, 'question_8_accuracy': 0.0, 'question_9_accuracy': 1.0, 'question_10_accuracy': 1.0, 'question_11_accuracy': 0.0, 'question_12_accuracy': 0.0, 'question_13_accuracy': 1.0, 'question_14_accuracy': 1.0, 'num_questions': 15, 'averaged_accuracy': 0.5333333333333333, 'num_valid_spans': 15}

{'finegrained_scores': [FinegrainedScore(score=0.0, effective_spans=[(0, 376)], reward_group_id=0, query_idx=None, response_idx=None, reward_group_name='question_0', advantage=None), FinegrainedScore(score=0.0, effective_spans=[(376, 611)], reward_group_id=0, query_idx=None, response_idx=None, reward_group_name='question_0', advantage=None), FinegrainedScore(score=1.0, effective_spans=[(611, 1010)], reward_group_id=0, query_idx=None, response_idx=None, reward_gr

In [10]:
from transformers import AutoTokenizer

# Load Qwen tokenizer
tokenizer = AutoTokenizer.from_pretrained("ai2-adapt-dev/tulu_3_long_finetune_qwen_7b_reg")
print(f"Loaded tokenizer: {tokenizer.__class__.__name__}")
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"EOS token: {tokenizer.eos_token}")
print(f"EOS token ID: {tokenizer.eos_token_id}")


/home/rulin/miniconda3/envs/open-instruct/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded tokenizer: Qwen2TokenizerFast
Vocab size: 151643
EOS token: <|endoftext|>
EOS token ID: 151643


In [99]:
token_resp = tokenizer(decoded_resp, return_tensors="pt", add_special_tokens=False)["input_ids"][0]
print(token_resp)

tensor([ 198,   27, 9217,  ...,   16,   20,  397])


In [100]:
# decoded_resp
all_spans = [score.effective_spans[0] for score in reward_results["finegrained_scores"]]
for span in all_spans:
    print(decoded_resp[span[0]:span[1]])



<answer1>\boxed{Phaungkaza Maung Maung's paternal grandfather is King Takayoung of Burma. King Takayoung and Aung San are Phaungkaza Maung Maung's paternal grandparents, with Aung San being his father's (U Nu) father, who later became an important Burmese political leader and is widely regarded as the Father of Burma following the country's independence in 1948.}</answer1>


<answer2>\boxed{Mary Stewart, Countess of Arran, was the daughter of Arthur, Duke of Monaco, a prominent British aristocrat and philanthropist. Arthur Stewart's birthday is January 8, 1922, making him Mary Stewart's father.</answer2>
 

<answer3>\boxed{The director of the film Les Temps D'effervescence, known as Les J3 in French, was Patrice Chéreau, who was born on March 15, 1934. On the other hand, Cat's Eye, a film from 1985, was directed by Patricia Rozema. Since Patrice Chéreau was born earlier than Patricia Rozema, Les Temps D'effervescence has the director who was born first compared to Cat's Eye (1985 Film

In [105]:
for span_index in range(len(all_spans)):
    print(decoded_resp[all_spans[span_index][0]:all_spans[span_index][1]])
    test_mask, _ = convert_string_span_to_token_span([all_spans[span_index]], decoded_resp, token_resp, tokenizer)
    test_mask_indices = torch.where(torch.tensor(test_mask))[0]
    test_token_spans = test_mask_indices.min().item(), test_mask_indices.max().item() + 1
    # assert test_token_spans[1] - test_token_spans[0] == test_mask_indices.shape[0]

    print(tokenizer.decode(token_resp[test_token_spans[0]:test_token_spans[1]]))
    print("===" * 25 )


<answer1>\boxed{Phaungkaza Maung Maung's paternal grandfather is King Takayoung of Burma. King Takayoung and Aung San are Phaungkaza Maung Maung's paternal grandparents, with Aung San being his father's (U Nu) father, who later became an important Burmese political leader and is widely regarded as the Father of Burma following the country's independence in 1948.}</answer1>

<answer1>\boxed{Phaungkaza Maung Maung's paternal grandfather is King Takayoung of Burma. King Takayoung and Aung San are Phaungkaza Maung Maung's paternal grandparents, with Aung San being his father's (U Nu) father, who later became an important Burmese political leader and is widely regarded as the Father of Burma following the country's independence in 1948.}</answer1


<answer2>\boxed{Mary Stewart, Countess of Arran, was the daughter of Arthur, Duke of Monaco, a prominent British aristocrat and philanthropist. Arthur Stewart's birthday is January 8, 1922, making him Mary Stewart's father.</answer2>
<answer2>\b

"\n<answer1>\x08oxed{Phaungkaza Maung Maung's paternal grandfather is King Takayoung of Burma. King Takayoung and Aung San are Phaungkaza Maung Maung's paternal grandparents, with Aung San being his father's (U Nu) father, who later became an important Burmese political leader and is widely regarded as the Father of Burma following the country's independence in 1948.}</"

In [84]:
test_mask_indices.shape[0]

97

In [ ]:
from typing import Dict, Any, List, Tuple

def convert_string_spans_to_token_spans(
    tokenized_input: Dict[str, Any],
    message_string_spans: List[Tuple[int, int]],
    target_message_span_index: int
) -> int:
    """
    Convert a list of string spans to a list of token indices.
    """

    # first, we need to obtain the local offset_mapping given the target_message_span_index
    message_start_token_index, message_end_token_index = tokenized_input["message_token_spans"][
        target_message_span_index
    ]
    message_start_string_index = tokenized_input["offset_mapping"][message_start_token_index][0]

    # Then we need to obtain the global message string spans and obtain the token spans: 
    # The input message_string_spans is a list of tuples, each tuple is a local string span in the original document

    output_token_spans = []
    for local_string_start, local_string_end in message_string_spans:
        global_string_start = local_string_start + message_start_string_index
        global_string_end = local_string_end + message_start_string_index
        
        current_token_start, current_token_end = None, None

        for token_index, (token_string_start, token_string_end) in enumerate(tokenized_input["offset_mapping"][message_start_token_index:message_end_token_index], start=message_start_token_index):
            
            if token_string_start >= global_string_start:
                if current_token_start is None:
                    current_token_start = token_index
                current_token_end = token_index + 1

            if token_string_end > global_string_end:
                break
        
        if current_token_start is not None and current_token_end is not None:
            output_token_spans.append((current_token_start, current_token_end))

    return output_token_spans


def convert_string_span_to_token_span(effective_spans, decoded_resp, token_resp, tokenizer):
    """
    Convert character spans to token spans and create a mask for training.
    Uses the span conversion logic from open_instruct/search_rewards/span_convertion.py
    
    Args:
        effective_spans: List of tuples (start_char, end_char) in decoded response
        decoded_resp: Decoded string response
        token_resp: Tokenized response (list of token IDs)
        tokenizer: Tokenizer used to decode the responses
    
    Returns:
        Tuple of (mask, span_mapping_stats) where:
        - mask: List of integers where 1 means the token should be trained on, 0 means masked
        - span_mapping_stats: Dict with mapping quality statistics
    """

    
    logger = logging.getLogger(__name__)
    
    # Initialize statistics tracking
    span_stats = {
        "total_spans": len(effective_spans) if effective_spans else 0,
        "valid_start_mappings": 0,
        "valid_end_mappings": 0,
    }
    
    if not effective_spans or len(token_resp) == 0:
        # If no effective spans or no tokens, mask everything except last EOS
        mask = [0] * len(token_resp)
        if len(token_resp) > 0 and token_resp[-1] == tokenizer.eos_token_id:
            mask[-1] = 1
        return mask, span_stats
    
    # Use the span conversion utilities directly without creating dummy messages
    # We'll tokenize the decoded response directly and get offset mappings
    tokenized_resp = tokenizer(
        decoded_resp,
        return_tensors="pt",
        return_offsets_mapping=True,
        add_special_tokens=False,
    )
    
    # Check for empty tokenization
    if len(tokenized_resp["input_ids"][0]) == 0:
        logger.warning(f"Empty tokenization for decoded_resp: '{decoded_resp[:100]}...'")
        # Return all-masked except EOS
        mask = [0] * len(token_resp)
        if len(token_resp) > 0 and token_resp[-1] == tokenizer.eos_token_id:
            mask[-1] = 1
        return mask, span_stats
    
    # Create a simple tokenized_input structure that the conversion function expects
    tokenized_input = {
        "offset_mapping": tokenized_resp["offset_mapping"][0],  # Remove batch dimension
        "message_token_spans": {0: [0, len(tokenized_resp["input_ids"][0])]},  # Single message spanning all tokens
    }
    
    # Convert string spans to token spans using the utility function
    token_spans = convert_string_spans_to_token_spans(
        tokenized_input,
        effective_spans,
        target_message_span_index=0
    )
    
    # Initialize mask - everything masked by default
    mask = [0] * len(token_resp)
    
    # Update statistics and create mask
    for span_idx, (token_start, token_end) in enumerate(token_spans):
        span_stats["valid_start_mappings"] += 1
        span_stats["valid_end_mappings"] += 1
        
        # Clamp to valid token range for our response
        token_start = max(0, min(token_start, len(token_resp)))
        token_end = max(token_start, min(token_end, len(token_resp)))
        
        # Unmask tokens in this span
        for token_idx in range(token_start, token_end):
            if token_idx < len(mask):
                mask[token_idx] = 1
    
    # Always keep the last EOS token unmasked if it exists
    if len(token_resp) > 0 and token_resp[-1] == tokenizer.eos_token_id:
        mask[-1] = 1
    
    return mask, span_stats

In [114]:
finegrained_score = reward_results["finegrained_scores"][0]
print(finegrained_score)
spans = finegrained_score.effective_spans
mask, span_stats = convert_string_span_to_token_span(
    effective_spans=spans,
    decoded_resp=decoded_resp,
    token_resp=token_resp,
    tokenizer=tokenizer
)
print(f"Number of unmasked tokens: {sum(mask)}")
print(f"Effective spans: {spans}")

# Convert mask to tensor and select tokens based on mask
import torch
mask_tensor = torch.tensor(mask, dtype=torch.bool)
selected_tokens = token_resp[mask_tensor]
print(f"Selected tokens shape: {selected_tokens.shape}")
print(f"Selected tokens: {selected_tokens}")
print(f"Decoded selected tokens:")
print(selected_tokens.shape)
print(len(mask))
print(token_resp.shape)
print(tokenizer.decode(token_resp) == decoded_resp)

FinegrainedScore(score=0.0, effective_spans=[(0, 376)], reward_group_id=0, query_idx=None, response_idx=None, reward_group_name='question_0', advantage=None)
Number of unmasked tokens: 98
Effective spans: [(0, 376)]
Selected tokens shape: torch.Size([98])
Selected tokens: tensor([   198,     27,   9217,     16,   8449,  79075,     90,   3357,     64,
          2185, 133821,  11331,   2185,  11331,   2185,    594,  92267,  37850,
           374,   6210,  33290,    352,    283,    968,    315,  91571,     13,
          6210,  33290,    352,    283,    968,    323,    362,   2185,   5836,
           525,   2350,     64,   2185, 133821,  11331,   2185,  11331,   2185,
           594,  92267,  55335,     11,    448,    362,   2185,   5836,   1660,
           806,   6981,    594,    320,     52,  32324,      8,   6981,     11,
           879,   2937,   6116,    458,   2989,  12369,     76,   2367,   4948,
          7653,    323,    374,  13570,  26361,    438,    279,  20322,    315,
       

In [115]:

tokenizer.decode(selected_tokens)


"\n<answer1>\\boxed{Phaungkaza Maung Maung's paternal grandfather is King Takayoung of Burma. King Takayoung and Aung San are Phaungkaza Maung Maung's paternal grandparents, with Aung San being his father's (U Nu) father, who later became an important Burmese political leader and is widely regarded as the Father of Burma following the country's independence in 1948.}</answer1>\n\n"

In [116]:
decoded_resp[spans[0][0]:spans[0][1]]

"\n<answer1>\\boxed{Phaungkaza Maung Maung's paternal grandfather is King Takayoung of Burma. King Takayoung and Aung San are Phaungkaza Maung Maung's paternal grandparents, with Aung San being his father's (U Nu) father, who later became an important Burmese political leader and is widely regarded as the Father of Burma following the country's independence in 1948.}</answer1>"